## 실습 [9-1]  

### 1. 실습명 : N-gram 언어 모델로 문장 생성하기
### 2. 실습 목적 및 설명
 
*   파이썬의 NLTK 패키지를 이용하여 N-gram 언어 모델을 구축한다
*   네이버에서 오픈 소스로 제공하는 nsmc 영화 리뷰 데이터셋을 이용해 문장을 생성한다.

### 3. 관련 장(챕터) : 9.2.2 N-gram 언어 모델 (N-gram Language Model)
### 4. 코드

In [1]:
import nltk
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
import numpy as np
import codecs
from tqdm import tqdm
import random

In [2]:
# 한국어 처리에 필요한 konlpy 패키지를 설치하기 전에 선행 파일을 설치한다. 
!apt-get update

!apt-get install g++ openjdk-8-jdk python-dev python3-dev

# allow python programs full access to java class libraries
!pip3 install JPype1-py3

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

from konlpy.tag import Okt

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://ppa

In [9]:
# NLTK 패키지를 이용하여 입력 텍스트를 N-gram 형태로 변환한다.
sentence = "나는 매일 아침 지하철을 탄다"

In [10]:
# NLTK 사용을 위하여 선행 패키지를 설치한다.
nltk.download('punkt')

# 입력 텍스트를 띄어쓰기 기준으로 토큰화한다.
tokens = word_tokenize(sentence) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
print(tokens)

['나는', '매일', '아침', '지하철을', '탄다']


In [12]:
# 한국어의 단어는 띄어쓰기를 기준으로 하지 않기 때문에 konlpy를 이용해 형태소를 기준으로 토큰화한다.
tagger = Okt() # 과거 트위터 형태소 분석기

def tokenize(text):
  tokens = ['/'.join(t) for t in tagger.pos(text)]
  return tokens

tokens = tokenize(sentence)
print(tokens)

['나/Noun', '는/Josa', '매일/Noun', '아침/Noun', '지하철/Noun', '을/Josa', '탄다/Verb']


In [17]:
# 토큰을 N-gram의 형태로 바꾸어준다. 
# ngrams 함수의 두 번째 인자로 N값을 지정할 수 있다.
# ngrams는 nltk.util 안에 있다 https://www.nltk.org/api/nltk.html#nltk.util.ngrams 
bigram = ngrams(tokens, 2) # Return type: sequence or iter
trigram = ngrams(tokens, 3)

In [18]:
# N-gram을 출력해본다.
print("bigram: ")
for b in bigram:
  print(b)

print("\ntrigram: ")
for t in trigram:
  print(t) 

bigram: 
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')

trigram: 
('나/Noun', '는/Josa', '매일/Noun')
('는/Josa', '매일/Noun', '아침/Noun')
('매일/Noun', '아침/Noun', '지하철/Noun')
('아침/Noun', '지하철/Noun', '을/Josa')
('지하철/Noun', '을/Josa', '탄다/Verb')


In [ ]:
# padding을 통해 입력 데이터에 문장의 시작과 끝을 알리는 토큰을 추가한다.
# https://www.nltk.org/api/nltk.html#nltk.util.ngrams  
bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
print("bigrams with padding: ")
for b in bigram:
  print(b)

bigram with padding: 
('<s>', '나/Noun')
('나/Noun', '는/Josa')
('는/Josa', '매일/Noun')
('매일/Noun', '아침/Noun')
('아침/Noun', '지하철/Noun')
('지하철/Noun', '을/Josa')
('을/Josa', '탄다/Verb')
('탄다/Verb', '</s>')


In [19]:
# 문장 생성을 위하여 네이버 영화 리뷰 데이터셋을 다운로드한다. (web get)
%%time
!wget -nc -q https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt # raw 텍스트 파일

CPU times: user 11.8 ms, sys: 14.3 ms, total: 26.2 ms
Wall time: 1.13 s


In [20]:
# 다운로드 받은 데이터셋을 읽고 인덱스와 라벨을 제외한 텍스트 부분만 가져온다.
# codecs 패키지는 대용량 파일을 조금씩 읽을 수 있게 해준다. 

with codecs.open("ratings_train.txt", encoding='utf-8') as f:
  data = [line.split('\t') for line in f.read().splitlines()] # \n 제외 -> [['id', 'document', 'label'], ['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ...]
  data = data[1:] # header 제외
print("데이터셋: ", data[:10])
docs = [row[1] for row in data] # 텍스트 부분만 가져옴
print("\n텍스트 데이터:", docs[:5])
print("\n문장 개수: ",len(docs)) # 총 15만개의 문장으로 이루어진 데이터셋임을 알 수 있다.

데이터셋:  [['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'], ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'], ['10265843', '너무재밓었다그래서보는것을추천한다', '0'], ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'], ['6483659', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', '1'], ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'], ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'], ['9443947', '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네', '0'], ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'], ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

텍스트 데이터: ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']

문장 개수:  150000


In [21]:
# 토큰화한 텍스트 데이터의 bigram을 모두 리스트에 추가한다.
sentences = []
for d in tqdm(docs): # https://github.com/tqdm/tqdm 
  tokens = tokenize(d)
  bigram = ngrams(tokens, 2, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>")
  sentences += [t for t in bigram]

100%|██████████| 150000/150000 [07:05<00:00, 352.22it/s]


In [22]:
print(sentences[:10])

[('<s>', '아/Exclamation'), ('아/Exclamation', '더빙/Noun'), ('더빙/Noun', '../Punctuation'), ('../Punctuation', '진짜/Noun'), ('진짜/Noun', '짜증나네요/Adjective'), ('짜증나네요/Adjective', '목소리/Noun'), ('목소리/Noun', '</s>'), ('<s>', '흠/Noun'), ('흠/Noun', '.../Punctuation'), ('.../Punctuation', '포스터/Noun')]


In [28]:
cfd = ConditionalFreqDist(sentences)
print(cfd["<s>"].most_common(2100))

[('정말/Noun', 2718), ('이/Noun', 2371), ('진짜/Noun', 2232), ('이/Determiner', 2115), ('영화/Noun', 2069), ('아/Exclamation', 1693), ('너무/Adverb', 1627), ('평점/Noun', 1583), ('내/Noun', 1542), ('최고/Noun', 1127), ('1/Number', 1100), ('그냥/Noun', 1068), ('이런/Adjective', 916), ('재미/Noun', 777), ('완전/Noun', 773), ('이건/Noun', 768), ('난/Noun', 748), ('나/Noun', 732), ('역시/Noun', 708), ('왜/Noun', 670), ('마지막/Noun', 654), ('와/Verb', 642), ('스토리/Noun', 639), ('감독/Noun', 630), ('뭐/Noun', 602), ('말/Noun', 592), ('솔직히/Adjective', 591), ('쓰레기/Noun', 585), ('감동/Noun', 583), ('이영화/Noun', 552), ('지금/Noun', 517), ('배우/Noun', 512), ('내용/Noun', 504), ('"""/Punctuation', 490), ('처음/Noun', 484), ('별로/Noun', 439), ('시간/Noun', 436), ('원작/Noun', 415), ('내/Determiner', 412), ('최악/Noun', 397), ('이런/Modifier', 394), ('좋은/Adjective', 369), ('10/Number', 359), ('굿/Noun', 358), ('별/Noun', 351), ('한국/Noun', 346), ('다시/Noun', 346), ('아이/Noun', 343), ('저/Noun', 330), ('오랜/Modifier', 324), ('어릴/Verb', 316), ('액션/Noun', 313), ('사랑/

In [29]:
# 주어진 토큰(c) 다음에 가장 많이 등장하는 n개의 단어를 반환하는 함수를 만든다.
def most_common(c, n, pos=None):
  if pos is None:
    return cfd[tokenize(c)[0]].most_common(n)
  else:
    return cfd["/".join([c, pos])].most_common(n)

In [30]:
print(most_common("나", 10))

[('는/Josa', 831), ('의/Josa', 339), ('만/Josa', 213), ('에게/Josa', 148), ('에겐/Josa', 84), ('랑/Josa', 81), ('한테/Josa', 50), ('참/Verb', 45), ('이/Determiner', 44), ('와도/Josa', 43)]


In [31]:
# 단어별 등장 빈도를 기반으로 조건부 확률을 추정한다.
cpd = ConditionalProbDist(cfd, MLEProbDist) # MLEProbDist from nltk.probability, type: string representation

In [ ]:
# “.” 다음에 “</s>”가 올 확률을 출력한다.
print(cpd[tokenize(".")[0]].prob("</s>")) # 중간에 ... 가 많아서 생각보다 높을 것으로 추정

0.39102658679807606


In [32]:
# 토큰 c 다음에 토큰 w가 bigram으로 함께 등장할 확률을 구한다. P(w|c)
def bigram_prob(c, w):
  context = tokenize(c)[0]
  word = tokenize(w)[0]
  return cpd[context].prob(word)

In [33]:
print(bigram_prob("이", "영화"))

0.4010748656417948


In [34]:
print(bigram_prob("영화", "이"))

0.00015767585785521414


In [ ]:
# 조건부 확률을 알게 되면 가장 확률이 높은 토큰열을 토대로 문장을 생성할 수 있다.
def generate_sentence(seed=None, debug=False):
  if seed is not None:
    import random
    random.seed(seed) # seed 같으면 동일한 순서로 난수 발생
  c = "<s>"
  sentence = []
  while True:
    if c not in cpd:
      break
    w = cpd[c].generate()

    if w == "</s>":
      break
    
    word = w.split("/")[0]
    pos = w.split("/")[1]

    # 조사, 어미 등을 제외하고 각 토큰은 띄어쓰기로 구분하여 생성한다.
    if c == "<s>":
      sentence.append(word.title())
    elif c in ["`", "\"","'","("]:
      sentence.append(word)
    elif word in ["'", ".", ",", ")", ":", ";", "?"]:
      sentence.append(word)
    elif pos in ["Josa", "Punctuation", "Suffix"]:
        sentence.append(word)
    elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
        sentence.append(word)
    else:
        sentence.append(" " + word)
    c = w

    if debug:
      print(w)

  return "".join(sentence)

In [ ]:
print(generate_sentence(2))

도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯


In [ ]:
generate_sentence(2, debug=True)

도리/Noun
까지/Josa
본/Verb
영화/Noun
너무/Adverb
.../Punctuation
뭔가/Noun
../Punctuation
최고/Noun
네/Suffix
요/Josa
./Punctuation
하지만/Conjunction
../Punctuation
눈물/Noun
낫다는건/Verb
또/Noun
영화/Noun
에/Josa
들지/Verb
않는다/Verb
./Punctuation
근데/Adverb
뭐/Noun
야/Josa
어떻게/Adjective
그렇게/Adverb
착했던/Adjective
윤재/Noun
랑은/Josa
에바/Noun
그린/Noun
드레스/Noun
소리/Noun
듣는거/Verb
임/Noun
"""/Punctuation
에리/Noun
욧의/Noun
미모/Noun
로/Josa
합성/Noun
한/Determiner
가수/Noun
노래/Noun
와/Josa
흥행/Noun
놓친/Verb
영화/Noun
다/Josa
./Punctuation
사투리/Noun
연기/Noun
하나/Noun
없는/Adjective
‘/Foreign
스피드/Noun
감/Noun
넘치는/Adjective
스릴/Noun
넘치는/Adjective
연기/Noun
를/Josa
이해/Noun
되지/Verb
못/VerbPrefix
하시는/Verb
분/Noun
보다/Josa
훨/Noun
재밌구만/Adjective
평점/Noun
을/Josa
망처/Noun
놓은/Verb
듯/Noun
하다/Verb
./Punctuation
영화/Noun
보는이로/Verb
하여금/Adverb
불편함을/Adjective
느꼇을듯/Noun


'도리까지 본 영화 너무... 뭔가.. 최고네요. 하지만.. 눈물 낫다는건 또 영화에 들지 않는다. 근데 뭐야 어떻게 그렇게 착했던 윤재랑은 에바 그린 드레스 소리 듣는거임""" 에리 욧의 미모로 합성 한 가수 노래와 흥행 놓친 영화다. 사투리 연기 하나 없는 ‘ 스피드 감 넘치는 스릴 넘치는 연기를 이해 되지 못 하시는 분보다 훨 재밌구만 평점을 망처 놓은 듯하다. 영화 보는이로 하여금 불편함을 느꼇을듯'

### 5. 결과
： 위 예제는 코퍼스 내의 등장 빈도에 기반하여 문장을 생성한다. bigram 언어 모델로 생성한 것이기 때문에 인접한 두 단어는 그나마 자연스럽지만 멀리 떨어진 단어와는 전혀 무관한 모습을 보인다. 또한 생성된 문장의 전체적인 문맥이 부자연스러우며 통사적으로 부적절한 모습도 보인다. 이는 코퍼스 내의 정보만으로 제한된 단어 조합만을 고려하는 N-gram 언어 모델의 한계로 보인다. 위 예제는 단순화를 위해 전처리와 규칙 처리를 최소화하였는데, 데이터셋을 늘리고 한국어 특징에 맞게 전처리를 진행한다면 보다 향상된 결과를 얻을 수 있을 것이다.


<참고>

> https://datascienceschool.net/view-notebook/a0c848e1e2d343d685e6077c35c4203b/

안 열리면:
>https://datascienceschool.net/03%20machine%20learning/03.01.05%20%ED%99%95%EB%A5%A0%EB%A1%A0%EC%A0%81%20%EC%96%B8%EC%96%B4%20%EB%AA%A8%ED%98%95.html 
